In [1]:
import sagemaker
import boto3
from datetime import datetime
from io import StringIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot

warnings.filterwarnings('ignore')



from pyathena import connect

In [3]:
sess = sagemaker.Session() #Intialzie session

bucket = sess.default_bucket()
# RD_Bucket = 'policedatasetbucket' #Raw Data
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
s3 = boto3.client('s3') # Create an S3 client
s3_resource = boto3.resource('s3') # Create an S3 resource


print('Bucket - > ',bucket)
print('Role - > ',role)
print('Region - > ',region)
print('S3 - > ',s3)
print('S3 Resource - > ',s3_resource)

Bucket - >  sagemaker-us-east-1-859074047513
Role - >  arn:aws:iam::859074047513:role/LabRole
Region - >  us-east-1
S3 - >  <botocore.client.S3 object at 0x7f5ab8142810>
S3 Resource - >  s3.ServiceResource()


In [4]:
# Function to verify if bucket exist, if not create
def verify_create_bucket(bucket_name):
    response = s3.list_buckets()
    for bucket in response['Buckets']:
        if bucket['Name'] == bucket_name:
            print(f"The {bucket_name} bucket exists.")
            break
    else:
        print(f"The {bucket_name} bucket does not exist, creating")
        !aws s3 mb s3://{bucket}/

In [ ]:
verify_create_bucket(bucket)

In [ ]:
#function to download yearly data
# def Get_Data(year):
#     url = f"https://seshat.datasd.org/pd/pd_calls_for_service_{year}_datasd.csv"
#     df = pd.read_csv(url)
#     return df

In [ ]:
# List of years since 2018
# Years = list(range(2018,datetime.now().year+1))
# Years

In [ ]:
#iterate over years
# for year in Years:
#     year_df = Get_Data(year)
#     csv_buffer = StringIO()
#     year_df.to_csv(csv_buffer, index=False)
#     file = 'SDPD_Calls_' + str(year) + '.csv'
#     s3_resource.Object(bucket,file).put(Body=csv_buffer.getvalue())
#     print(file,'loaded in',bucket,'bucket')

In [ ]:
# def URL_2_Bucket(url,file_name):
#     df = pd.read_csv(url, low_memory=False)
#     csv_buffer = StringIO()
#     df.to_csv(csv_buffer, index=False)
#     file = file_name + '.csv'
#     s3_resource.Object(RD_Bucket,file).put(Body=csv_buffer.getvalue())
#     print(file,'loaded in',RD_Bucket,'bucket')
    

In [ ]:
# type_url = f"http://seshat.datasd.org/pd/pd_cfs_calltypes_datasd.csv"
# dipo_url = f"http://seshat.datasd.org/pd/pd_dispo_codes_datasd.csv"
# ripa_stops_url = "https://seshat.datasd.org/pd/ripa_stops_datasd.csv"
# ripa_stops_dic = "https://seshat.datasd.org/pd/ripa_stops_dictionary_datasd.csv"

In [ ]:
# URL_2_Bucket(type_url,'Type')
# URL_2_Bucket(dipo_url,'Dispo')
# URL_2_Bucket(ripa_stops_url,'Ripa_Stops')
# URL_2_Bucket(ripa_stops_dic,'Ripa_Stops_Dic')

In [ ]:
ingest_create_athena_db_passed = False

# Create Athena Database

In [ ]:
database_name = "sd_police_db"

In [ ]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [ ]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)


In [ ]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

In [ ]:
import pandas as pd

pd.read_sql(statement, conn)

In [ ]:
# Show databases
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

# Drop Database if Needed

In [ ]:
# drop_db_name = "dsoaw"
# drop_db_query = f"DROP DATABASE IF EXISTS {drop_db_name}"

# df_show = pd.read_sql(drop_db_query, conn)
# df_show.head()

# Drop Table if Needed

In [ ]:
# drop_table_name = "table_sd_ripa" 

# drop_table_query = f"DROP TABLE IF EXISTS {database_name}.{drop_table_name}"

# df_show1 = pd.read_sql(drop_table_query, conn)
# df_show1.head()

In [ ]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [ ]:
%store ingest_create_athena_db_passed

# S3 Bucket Folder Information for 2021

In [ ]:
s3_private_path_csv = "s3://{}/SanDiego-Police-Dataset/policedatasetsd/csv".format(bucket)
print(s3_private_path_csv)

In [ ]:
table_name = "table_sd_2021"

In [ ]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
incident_num string,
date_time string,
day_of_week int,
address_number_primary int,
address_dir_primary string,
address_road_primary string,
address_sfx_primary string,
address_dir_intersecting string,
address_road_intersecting string,
address_sfx_intersecting string,
call_type string,
disposition string,
beat int,
priority int) 

ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' NULL DEFINED AS '' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(database_name, table_name, s3_private_path_csv)

print(statement)

In [ ]:
import pandas as pd

pd.read_sql(statement, conn)

# Varify Tables in a Specified Database

In [ ]:
database_name = "sd_police_db"
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)

df_show.head(5)

In [ ]:
pd.read_sql(statement, conn)

df_2021 = pd.read_sql(f"SELECT * FROM {database_name}.{table_name}", conn)

# S3 Bucket Folder Information for 2022

In [ ]:
s3_private_path_csv1 = "s3://{}/SanDiego-Police-Dataset/policedatasetsd22/csv".format(bucket)
print(s3_private_path_csv1)

In [ ]:
table_name1 = "table_sd_2022"

In [ ]:
# SQL statement to execute
statement_1 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
incident_num string,
date_time string,
day_of_week int,
address_number_primary int,
address_dir_primary string,
address_road_primary string,
address_sfx_primary string,
address_dir_intersecting string,
address_road_intersecting string,
address_sfx_intersecting string,
call_type string,
disposition string,
beat int,
priority int) 

ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' NULL DEFINED AS '' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(database_name, table_name1, s3_private_path_csv1)

print(statement_1)

In [ ]:
import pandas as pd

pd.read_sql(statement_1, conn)

In [ ]:
database_name = "sd_police_db"
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

In [ ]:
pd.read_sql(statement_1, conn)

df_2022 = pd.read_sql(f"SELECT * FROM {database_name}.{table_name1}", conn)

In [ ]:
df_2022

# S3 Bucket Folder Information for 2023

In [ ]:
s3_private_path_csv2 = "s3://{}/SanDiego-Police-Dataset/policedatasetsd23/csv".format(bucket)
print(s3_private_path_csv2)

In [ ]:
table_name2 = "table_sd_2023"

In [ ]:
# SQL statement to execute
statement_2 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
incident_num string,
date_time string,
day_of_week int,
address_number_primary int,
address_dir_primary string,
address_road_primary string,
address_sfx_primary string,
address_dir_intersecting string,
address_road_intersecting string,
address_sfx_intersecting string,
call_type string,
disposition string,
beat int,
priority int) 

ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' NULL DEFINED AS '' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1',
                'serialization.null.format'='')""".format(database_name, table_name2, s3_private_path_csv2)

print(statement_2)

In [ ]:
import pandas as pd

pd.read_sql(statement_2, conn)

In [ ]:
database_name = "sd_police_db"
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

In [ ]:
pd.read_sql(statement_2, conn)

df_2023 = pd.read_sql(f"SELECT * FROM {database_name}.{table_name2}", conn)

In [ ]:
df_2023

# S3 Bucket Folder Information for Police Call Type

In [ ]:
s3_private_path_csvcall = "s3://{}/SanDiego-Police-Dataset/policecalltypes/csv/".format(bucket)
print(s3_private_path_csvcall)

In [ ]:
table_name_call = "table_call_type"

In [ ]:
# SQL statement to execute
statement_calltype = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
call_type string,
description string
) 

ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' NULL DEFINED AS '' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1',
                'serialization.null.format'='')""".format(database_name, table_name_call, s3_private_path_csvcall)

print(statement_calltype)

In [ ]:
import pandas as pd

pd.read_sql(statement_calltype, conn)

In [ ]:
pd.read_sql(statement_calltype, conn)

df_call = pd.read_sql(f"SELECT * FROM {database_name}.{table_name_call}", conn)

In [ ]:
df_call

# Join The Three datasets 2021, 2022, 2023

In [ ]:
police_call_df = pd.read_sql(f"SELECT * FROM {database_name}.{table_name} UNION ALL \
                               SELECT * FROM {database_name}.{table_name1} UNION ALL \
                               SELECT * FROM {database_name}.{table_name2}", conn)

In [ ]:
display(police_call_df)

### Checking the data types, removing missing values, duplicated values

In [ ]:
# shape of the dataframes
print(f'shape of the dataframe police dataframe {df_2021.shape}')
print(f'shape of the dataframe police dataframe {df_2022.shape}')
print(f'shape of the dataframe police dataframe {df_2023.shape}')

In [ ]:
police_call_df.describe(include='all')

In [ ]:
# Data types 
police_call_df.dtypes

In [ ]:
from datetime import datetime

police_call_df["date_time"] = pd.to_datetime(police_call_df["date_time"])
police_call_df

In [ ]:
police_call_df.dtypes

In [ ]:
police_call_df.info()

Our first file contains 1,168,014 rows and 14 columns.</br>
Out of 14 columns 4 are int type, 8 are strings/object, and 1 is datetime type.</br>
Out of 14 columns 8 columns have null/missing values. 


In [ ]:
# check null values
police_call_df.isna().sum()

In [ ]:
from tqdm import tqdm

def Data_Quality_Report(df):

    #Initial table
    freqDF = pd.DataFrame(columns=['Feature',
                                   'Mode',
                                   'Mode Freq.',
                                   'Mode %',
                                   '2nd Mode',
                                   '2nd Mode Freq.',
                                   '2nd Mode %'])
    for col in tqdm(df.columns):
        try:
            #print(col)
            freq = df[col].value_counts()
            freqdf = freq.to_frame()
            fRow = freqdf.iloc[0]
            #try:
            secRow = freqdf.iloc[1]
            #except:
            #secRow = 0
            fPrct = fRow[0] / len(df[col])
            #try:
            secPrct = secRow[0] / len(df[col])
            #except:
                #secPrct = 0
            try:
                mode1 = int(fRow.name)
            except:
                mode1 = fRow.name
            try:
                mode2 = int(secRow.name)
            except:
                try:
                    mode2 = secRow.name
                except:
                    mode2 = 0
            freqDF = freqDF.append({'Feature':col,
                                    'Mode':mode1,
                                    'Mode Freq.':fRow[0],
                                    'Mode %':fPrct,\
                                    '2nd Mode':mode2,
                                    '2nd Mode Freq.':secRow[0],
                                    '2nd Mode %':secPrct},
                                    ignore_index=True)
        except:
            pass

    freqDF = freqDF.set_index('Feature')

    #Nulls, Counts, Cardinality
    NUllFeatures = round(df.isnull().sum() / df.shape[0],4)\
          .sort_values(ascending=False)
    Count = df.count()
    uni = df.nunique()

    #Formating
    NUllFeatures.to_frame(name="% Miss.")
    Count.to_frame(name="Count")
    uni.to_frame()
    result = pd.concat([Count, NUllFeatures,uni], axis=1)
    result.columns =["Count","% Miss.","Card."]
    result = pd.concat([result, freqDF], axis=1)
    """
    result = result.style.format({'% Miss.': "{:.1%}",
                         'Mode %': "{:.0%}",
                         '2nd Mode %': "{:.0%}",
                         'Count': "{:,}",
                         'Card.': "{:,}",
                         'Mode Freq.': "{:,}",
                        '2nd Mode Freq.': "{:,}"})"""
    return result

# Data Cleaning 

In [ ]:
# DQR_Calls = Data_Quality_Report(join_df)
# DQR_Calls

As we can see the `address_sfx_intersecting` and `address_dir_intersecting` have no values and we can drop these columns.</br>
Also the `address_dir_primary` and `address_road_intersecting` have more than 80% missing values; therefore we can consider removing these columns too. 

#### Dropping columns with more than 60% missing values

In [ ]:
# Removing unnecessary columns
def drop_columns_with_max_missing_values(df):
    mis_var = [x for x in df.columns if df[x].isnull().sum() > 0]
    df[mis_var].isnull().sum()

    limit = np.abs((df.shape[0] * 0.6))
    var_to_be_dropped = [x for x in mis_var if df[x].isnull().sum() > limit]
    print('Columns with more than 60% missing values: \n\n', var_to_be_dropped)

    df.drop(columns=var_to_be_dropped, axis=1, inplace=True)
    return df

df = drop_columns_with_max_missing_values(police_call_df)

In [ ]:
police_call_df.drop(["incident_num"],axis=1,inplace=True)

In [ ]:
police_call_df.shape

We dropped the unnecesssary columns from the dataset

In [ ]:
# check duplicated values
police_call_df.duplicated().sum()

We don't have any duplicated values in dataset

In [ ]:
police_call_df.isnull().sum()

In [ ]:
# Extract date and time components into separate columns
police_call_df["date"] = police_call_df["date_time"].dt.date
police_call_df["time"] = police_call_df["date_time"].dt.time

In [ ]:
police_call_df.drop(["date_time"],axis=1,inplace=True)

In [ ]:
police_call_df.head(10)

In [ ]:

police_call_df['date'] = pd.to_datetime(police_call_df['date'],format='%Y-%m-%d')

police_call_df["month"] = police_call_df["date"].dt.month
police_call_df["year"] = police_call_df["date"].dt.year
police_call_df

In [ ]:
# Change the day_of_week column to their specific day

# Replace the numbers with day names
day_mapping = {
    1: 'Sunday',
    2: 'Monday',
    3: 'Tuesday',
    4: 'Wednesday',
    5: 'Thursday',
    6: 'Friday',
    7: 'Saturday'
}

police_call_df['day_of_week'] = police_call_df['day_of_week'].replace(day_mapping)

In [ ]:
police_call_df

In [ ]:
# Convert 'time' column to datetime.time objects
police_call_df['time'] = pd.to_datetime(police_call_df['time'], format='%H:%M:%S').dt.time

def time_to_minutes(t):
    return t.hour * 60 + t.minute

# Apply the function to the time column
police_call_df['minutes'] = police_call_df['time'].apply(time_to_minutes)

# Define bin edges in mins
bin_range = [i * 60 for i in range(0, 25, 1)]

# Define bin labels 
bin_labels = [f"{i:02d}-{i+1:02d}" for i in range(0, 24)]

police_call_df['time_group'] = pd.cut(police_call_df['minutes'], bins=bin_range, labels=bin_labels, right=False)

police_call_df.drop(columns=['minutes'],axis=1, inplace=True)

In [ ]:
police_call_df.sort_values(by="date",ascending=True)

In [ ]:
# Adding a new column based on the time_group
def time_group_category(time_group):
    start_hr = int(time_group.split('-')[0])
    
    if 6 <= start_hr < 12:
        return 'Morning'
    elif 12 <= start_hr < 18:
        return 'Afternoon'
    elif 18 <= start_hr < 22:
        return 'Evening'
    else:
        return 'Night'

police_call_df["time_cat"] = police_call_df["time_group"].apply(time_group_category)

In [ ]:
police_call_df.drop(["time_group"],axis=1,inplace=True)

In [ ]:
# Converting the month col values to their specific month such as 1=January, 2=February, and so on
month_map = {1: 'January', 2: 'February', 3: 'March', 4: 'April',
             5: 'May', 6: 'June', 7: 'July', 8: 'August',
             9: 'September', 10: 'October', 11: 'November', 12: 'December'}

police_call_df["month"] = police_call_df["month"].map(month_map)

In [ ]:
police_call_df

In [ ]:
# Remove rows that contain missing values
police_call_df = police_call_df.dropna(axis=0)

In [ ]:
police_call_df.isnull().sum()

### Join Data Call type Dataframe to the Main Police Call Dataframe

In [ ]:
# Merge DataFrames 
main_df = police_call_df.merge(df_call,
                                  left_on='call_type',
                                  right_on='call_type',
                                  how='left')

In [ ]:
main_df.isnull().sum()

In [ ]:
missing_rows = main_df[main_df['description'].isnull()]
missing_rows

So we can we have some error types here and we're going to replace them with their correct stored values. 

In [ ]:
dic_errors = {'1016P':'1016PT', 'DRAGNET-': 'DRAGNET', 'found':'FOUND', 'plate':'PLATE', '211a': '211A'}

In [ ]:
main_df_1= main_df.replace(dic_errors)

In [ ]:
main_df_1.drop(["description"],axis=1,inplace=True)

In [ ]:
merged_main_df = main_df_1.merge(df_call,
                                  left_on='call_type',
                                  right_on='call_type',
                                  how='left')

In [ ]:
merged_main_df.isnull().sum()

In [ ]:
merged_main_df

### Converting the Tagret (`Priority`) variable to Three Different Groups ( `Distpatch Immediately`, `Distpatch`, `Non-urgent`) 

***Priority Zero***: Dispatch Immediately. Priority E calls involve an imminent
threat to life. Examples include: officer or person down, no detail accidents and
attempted suicide.

***Priority On***e: Dispatch Immediately. Priority One calls involve serious crimes
in progress or a threat to life. Examples include: missing children, child abuse,
domestic violence, disturbances involving weapons/violence and bomb threats.

***Priority Two***: Dispatch as quickly as possible. Priority Two calls involve
complaints regarding less serious crimes in which there is no threat to life.
Examples include: prowlers who have left, preserve the peace, crime reports for
residents standing by at an inconvenient location, blocked driveway when the
caller is waiting to leave, injured animals, loud parties with mitigating
circumstances.

***Priority Three***: Dispatch as quickly as possible. Priority Three calls involve
minor crimes or requests for service which are not urgent. Examples include:
investigating a cold crime, loud parties involving noise only.

***Priority Four***: Dispatch when no higher priority calls are waiting. Priority
Four calls involve minor requests for police service. Examples include: found
property, most parking violations, etc.

***Priority Nine***: Calls that are formatted for the Telephone Report Unit (TRU).
Priority Nine calls involve requests for police service, which have limited or no
suspect information. Examples include: petty theft, vehicle break-in and false
use of another’s identity.

The dispatch priority system is designed to be only a guide as a higher or lower
priority may be more appropriate.

In [ ]:
merged_main_df["priority"] = merged_main_df["priority"].astype("category")

In [ ]:
merged_main_df["priority"].value_counts()

Since we don't have priority `5` in our dictionary and it's not been interpreted, we can consider removing from the dataset. 

In [ ]:
priority_5 = merged_main_df[merged_main_df["priority"]==5]
priority_5

In [ ]:
# Dropping the priority 5
merged_main_df.drop(merged_main_df[merged_main_df["priority"]==5].index,inplace=True) 

In [ ]:
merged_main_df

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

merged_main_df.groupby("priority").count()["day_of_week"].plot(
    kind="bar", title="Breakdown by Priority Rating")

plt.xlabel("Priority")
plt.ylabel("Incident Count")

In [ ]:
# Change the `priority` values to categorical

# create a dictionary to map old values to new values
priority_map = {0: 'Dispatch immediately', 1: 'Dispatch immediately', 2: 'Dispatch', 3: 'Dispatch', 4: 'non-urgent', 9: 'non-urgent'}

merged_main_df["priority"] = merged_main_df["priority"].replace(priority_map)
merged_main_df

In [ ]:
counts_priority = merged_main_df["priority"].value_counts()
counts_priority

In [ ]:
merged_main_df.groupby("priority").count()["day_of_week"].plot(
    kind="bar", title="Breakdown by Priority Rating")

plt.xlabel("Priority")
plt.ylabel("Incident Count")

## Save the File Into the Bucket

In [ ]:
def move_2_bucket(bucket,subfolder_name,subfolder_name1,subfolder_name2,file_name,df):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3_resource.Object(bucket,
                       subfolder_name + subfolder_name1 + subfolder_name2 + file_name).put(Body=csv_buffer.getvalue())
    print(file_name,'loaded in',bucket,'bucket')

In [ ]:
move_2_bucket(bucket,'SanDiego-Police-Dataset/','Main_Cleaned_File/','csv/','main_cleaned.csv',merged_main_df)